#FASE 4
ETL PARA EXTRAÇÃO DE DADOS DO PETRÓLEO E ALIMENTAÇÃO EM BANCO

##Inicialização

In [5]:
import psycopg2
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [6]:
# prompt: CONECTAR EM BANCO DE DADOS POSTGRES DA PLATAFORMA RENDER E EXECUTAR UMA QUERY

# Conexão com o banco de dados
conn = psycopg2.connect(
    host="dpg-cocqqgmv3ddc739kbg3g-a.oregon-postgres.render.com",
    database="db_advz",
    user="postech",
    password='HcoCeCx2iXUk0DqR4Rny1sBlKEFJd0TA'
)

# Cria um cursor para executar as queries
cur = conn.cursor()


OperationalError: connection to server at "dpg-cocqqgmv3ddc739kbg3g-a.oregon-postgres.render.com" (35.227.164.209), port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?


##ETL

Aqui vamos extrair os dados da url e gerar um dataframe com os mesmos

In [5]:
# URL da página que contém a tabela
url = 'http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view'

# Fazendo o request para a página
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Encontrar a tabela pela ID
table = soup.find('table', id='grd_DXMainTable')

# Verifica se a tabela foi encontrada
if not table:
    print("Não foi possível encontrar a tabela com o ID especificado.")
    exit()

# Criar uma lista para armazenar os dados
data = []

# Extrair as linhas da tabela
rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols if ele.text.strip()]  # Remover espaços e valores vazios
    if cols and len(cols) >= 2:  # Verificar se a lista tem pelo menos dois elementos
        data.append(cols[:2])  # Apenas as duas primeiras colunas são necessárias

# Converter a lista em um DataFrame
df = pd.DataFrame(data, columns=['data', 'valor'])

# Removendo possíveis cabeçalhos repetidos nos dados
df = df[df['valor'].str.replace('.', '').str.replace(',', '').str.isdigit()]

# Conversão da coluna 'Valor' para o formato numérico correto (com ponto decimal)
df['valor'] = df['valor'].str.replace(',', '.').astype(float)

# Converter a coluna 'Data' em datetime
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

df


,data,valor
1,2024-05-03,83.60
2,2024-05-02,84.81
3,2024-05-01,83.55
4,2024-04-30,88.23
5,2024-04-29,88.44
...,...,...
11160,1987-05-26,18.63
11161,1987-05-25,18.60
11162,1987-05-22,18.55
11163,1987-05-21,18.45


Vamos fazer um select dos mesmos dados no nosso banco POSTGRES na plataforma cloud Render

In [6]:
cur.execute("SELECT * FROM ipea;")
rows = cur.fetchall()
dfipea = pd.DataFrame(rows)

dfipea.columns=['data','valor']

dfipea['data'] = pd.to_datetime(dfipea['data'], infer_datetime_format=True)
dfipea

<ipython-input-6-c6c22f41617e>:7: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dfipea['data'] = pd.to_datetime(dfipea['data'], infer_datetime_format=True)


,data,valor
0,2024-04-08,91.73
1,2024-04-05,92.81
2,2024-04-04,90.32
3,2024-04-03,90.6
4,2024-04-02,87.63
...,...,...
11145,2024-04-15,90.84
11146,2024-04-12,93.12
11147,2024-04-11,91.49
11148,2024-04-10,91.57


Vamos comparar os dados entre eles para minimizar o trabalho do banco e inserir apenas os novos dados

In [7]:
dfipea_recente = dfipea['data'].max()
df_novosdados = df[df['data'] > dfipea_recente]

In [8]:
df_novosdados

,data,valor
1,2024-05-03,83.60
2,2024-05-02,84.81
3,2024-05-01,83.55
4,2024-04-30,88.23
5,2024-04-29,88.44
6,2024-04-26,89.95
7,2024-04-25,88.10
8,2024-04-24,89.02
9,2024-04-23,88.29
10,2024-04-22,87.30


Inserção em si

In [9]:
try:
    for index, row in df_novosdados.iterrows():
        sql = "INSERT INTO ipea (data, valor) VALUES (%s, %s);"
        cur.execute(sql, (row['data'], row['valor']))

    conn.commit()
    print("Inserções bem-sucedidas.")
except Exception as e:
    conn.rollback()
    print("Erro durante as inserções:", e)
finally:
    conn.close()


Inserções bem-sucedidas.
